In [188]:
import re
import pickle
import Stemmer
import numpy as np
from collections import defaultdict, OrderedDict
import sys

In [189]:
re.split(':','b:Sachin c:world cup')

['b', 'Sachin c', 'world cup']

In [190]:
STOP_DICT = {}
# INPUT_FILE = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'
INDEX_DIR = 'index/'
QUERY = 'b:cristiano c:football'
if INDEX_DIR[-1]!='/':
    INDEX_DIR+='/'
STOP_DICT = {}
STOP_FILE = ''
if INDEX_DIR.split('/')[0] == '2018114017':
    STOP_FILE = '2018114017/stopwords.pickle'
else:
    STOP_FILE = 'stopwords.pickle'
with open(STOP_FILE, 'rb') as handle:
    STOP_DICT = pickle.load(handle)
handle.close()
stemmer = Stemmer.Stemmer('english')
f = open(INDEX_DIR+"index.txt", "r")
index_string = f.read()
f.close()
temp = index_string.split('\n')
index = defaultdict(list)
for i in temp:
    splits = i.split(':')
    index[splits[0]] = splits[1].split(' ')

In [191]:
def preprocess(text):
    text = text.lower()
    tokens = re.sub(r'[^A-Za-z0-9]+', r' ', text).split()
    stemmed_stop_free = []
    for token in tokens:
        if token not in STOP_DICT:
            stemmed_stop_free.append(stemmer.stemWord(token))
    return stemmed_stop_free

In [192]:
def parse_query(query):
    query_list = query.split(':')
    if len(query_list) == 1:
        return preprocess(query_list[0]) , 0
    else:
        query_dict = {}
        # query_dict[query_list[0]] = ''
        for i in range(1, len(query_list)-1):
            query_dict[query_list[i-1][-1]] = preprocess(query_list[i][:-2])
        query_dict[query_list[-2][-1]] = preprocess(query_list[-1])
        return query_dict, 1

In [193]:
def run_whole_query(query):
    docs_intersect = []
    docs_postlist = {}
    for i in range(len(query)):
        doc_list = []
        post = []
        for j in index[query[i]]:
            splits = re.split('d|b|i|l|r|t|c', j)
            doc_list.append(splits[1])
            post.append(j)
        docs_postlist[query[i]]=post
        if i == 0:
            docs_intersect = doc_list
        else:
            docs_intersect = np.intersect1d(docs_intersect, doc_list)
    return docs_intersect, docs_postlist

In [194]:
def run_parsed_query(query_dict):
    flag = 0
    docs_intersect = []
    docs_postlist = {}
    for category in query_dict.keys():
        for i in range(len(query_dict[category])):
            doc_list = []
            post = []
            for j in index[query_dict[category][i]]:
                if category == 't':
                    splits = re.split('t', j)
                if category == 'b':
                    splits = re.split('b', j)
                if category == 'i':
                    splits = re.split('i', j)
                if category == 'c':
                    splits = re.split('c', j)
                if category == 'l':
                    splits = re.split('l', j)
                if category == 'r':
                    splits = re.split('r', j)
                if len(splits) > 1:
                    post.append(j)
                    splits2 = re.split('d|b|i|l|r|t|c', j)
                    doc_list.append(splits2[1])
                    docs_postlist[category+':'+query_dict[category][i]]=j
            docs_postlist[category+':'+query_dict[category][i]]=post
            if flag == 0:
                docs_intersect = doc_list
                flag = 1
            else:
                docs_intersect = np.intersect1d(docs_intersect, doc_list)
    return docs_intersect, docs_postlist

In [195]:
def print_postlist(postlist, docs_ids):
    for i in postlist.keys():
        print("Postlist for",i+":")
        for j in postlist[i]:
            print(j,end=' ')
        print()
        print()

In [196]:
parsed, querytype = parse_query(QUERY)
docs_intersect = []
docs_postlist = {}
if querytype == 0:
    docs_intersect, docs_postlist = run_whole_query(parsed)
else:
    docs_intersect, docs_postlist = run_parsed_query(parsed)
print_postlist(docs_postlist, docs_intersect)

Postlist for b:cristiano:
d1493b1i1 d1870b1 d3389b1 d3454b3 d3526b1 d4152b1 d5318b1 d5342b1 d6285b1 d6694b4 d6989b1 d7066b4 d8497b1 d9244b1 d10088b1r1 d10095b1 d10184b1 d11926b1 d12897b5 d12939b1 d14597b1 d14983b4 d15568b1 d15606b1 d15788b2i2 d17270b9i1r4 d17357b2 d17435b1 d17968b1 d18049b1 d18393b1 d19335b1 

Postlist for c:footbal:
d107b16i28c2r1 d1107b46i3c3l1r1 d1108b66i3c3l1r1 d1157b152i6c4l1r10 d1222b85i3c4r3 d1234b22i1c4 d1276b118i3c2l6r4 d1277b38i4c4r7 d1305t1b207i10c2l5r6 d1531b2c2r1 d1826b25i4c3l2 d2298b2c1 d2394t1b53i14c14r2 d2437b69i6c11l2r3 d2478b92i3c3l1r2 d2489b53i13c5l1r7 d2544b169i5c5r1 d2883b46i7c7l5r7 d2884b15c3r1 d2975b70i75c4l2r1 d3486c2r1 d3996b56i11c2l1r1 d4021b31i2c3l1r3 d4022b62i2c3l2r2 d4023b36i4c3l1r1 d4046b34i1c3l1r14 d4095b126i5c5r3 d4129t1b464c1l11r26 d4201b38i2c12 d4217b19i4c3r1 d4258t1b103i9c2l2r5 d4568b82i5c4r5 d5001b31i3c2l1r1 d5002b44i3c4l1r3 d5017b40i5c7r2 d5034b5c2 d5235b119i11c19r5 d5293b56i3c4l1r1 d5318b102i7c12l1r15 d5339t1b12c3l2 d5591b30c2r1 d6

In [197]:
# docs_postlist, parsed